# Baseline experiments

Импорт либ

In [1]:
# !pip install --pre torch torchvision torchaudio --index-url https://download.pytorch.org/whl/nightly/cu126

In [1]:
import gdown
import zipfile
import os
import pandas as pd
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from tqdm import tqdm
import cv2 as cv
import seaborn as sns
from itertools import chain
from sklearn.preprocessing import label_binarize
from matplotlib import colors
from mpl_toolkits.mplot3d import Axes3D
from typing import List, Tuple
import random
import torch
from torchvision.transforms import ToTensor
from urllib.request import urlopen
import timm
from torchvision.datasets import ImageFolder
from torch.utils.data import Dataset, DataLoader
from sklearn.svm import SVC
import pickle
import torch.nn as nn
import torch.nn.functional as F
from sklearn.metrics import roc_auc_score, roc_curve, auc
from catboost import CatBoostClassifier

### EfficientNetV2-S (input 450x450)

In [1]:
imagefolder = "C:/Users/User/images/450/"

Feature Extraction

In [43]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Using device:", device)

model = timm.create_model(
    'tf_efficientnetv2_s.in21k_ft_in1k',
    pretrained=True,
    features_only=True,
)

model = model.to(device)
model.eval()
data_config = timm.data.resolve_model_data_config(model)
transforms = timm.data.create_transform(**data_config, is_training=False)

def extract_features(images):
    with torch.no_grad():
        outputs = model(images)
        last_feature_map = outputs[-1]  # Get the last feature map
        # Flatten the feature map
        features = last_feature_map.view(last_feature_map.size(0), -1)
        return features

# Создаём DataLoader для последующего Feature extraction'a
dataset = ImageFolder(imagefolder, transform=transforms)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

sample_images, _ = next(iter(dataloader))
sample_images = sample_images.to(device)
sample_features = extract_features(sample_images)
feature_size = sample_features.size(1)

num_samples = len(dataset)
X = torch.zeros(num_samples, feature_size)
y = torch.zeros(num_samples, dtype=torch.long)

start_idx = 0
for images, labels in tqdm(dataloader):
    images = images.to(device)
    labels = labels.to(device)

    features = extract_features(images)
    batch_size = features.size(0)

    # Move features and labels to CPU and store them
    X[start_idx:start_idx+batch_size] = features.cpu()
    y[start_idx:start_idx+batch_size] = labels.cpu()

    start_idx += batch_size

train_size = round(train_test_ratio*len(X))
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]
print(f"Train size is {train_size}")
print(X_train.shape, X_test.shape)
print(y_train.shape, y_test.shape)

Using device: cuda


Unexpected keys (bn2.bias, bn2.num_batches_tracked, bn2.running_mean, bn2.running_var, bn2.weight, classifier.bias, classifier.weight, conv_head.weight) found while loading pretrained weights. This may be expected if model is being adapted.
100%|██████████████████████████████████████████████████████████████████████████████| 1039/1039 [05:10<00:00,  3.35it/s]

Train size is 26586
torch.Size([26586, 25600]) torch.Size([6646, 25600])
torch.Size([26586]) torch.Size([6646])


In [6]:
# Экспорт фичей
X_train_numpy, X_test_numpy = X_train.numpy(), X_test.numpy()
y_train_numpy, y_test_numpy = y_train.numpy(), y_test.numpy()

np.savetxt("X_train_V2_S_450.csv", X_train_numpy, delimiter=",")
np.savetxt("X_test_V2_S_450.csv", X_test_numpy, delimiter=",")
np.savetxt("y_train_V2_S_450.csv", y_train_numpy, delimiter=",")
np.savetxt("y_test_V2_S_450.csv", y_test_numpy, delimiter=",")

In [2]:
# Импорт фичей
X_train_numpy = np.genfromtxt("X_train_V2_S_450.csv", delimiter=",")
X_test_numpy = np.genfromtxt("X_test_V2_S_450.csv", delimiter=",")
y_train_numpy = np.genfromtxt("y_train_V2_S_450.csv", delimiter=",")
y_test_numpy = np.genfromtxt("y_test_V2_S_450.csv", delimiter=",")

In [3]:
print(X_train_numpy.shape)
print(X_test_numpy.shape)
print(y_train_numpy.shape)
print(y_test_numpy.shape)

(26586, 1280)
(6646, 1280)
(26586,)
(6646,)


In [4]:
# Здесь эскпериментируем с модельками
model = CatBoostClassifier(
    iterations=100, 
    loss_function='MultiClass', 
    verbose=False,
    task_type='GPU',
    devices='0'
    )

grid = {'learning_rate': [0.01, 0.05, 0.1],
        'depth': [4, 6, 8, 10],
        'l2_leaf_reg': [1, 3, 5, 7]
       }

grid_search_result = model.grid_search(grid,
                                       X=X_train_numpy,
                                       y=y_train_numpy,
                                       refit=False
                                      )

model = CatBoostClassifier(
    iterations=5000, 
    loss_function='MultiClass', 
    verbose=True,
    task_type='GPU',
    devices='0',
    **grid_search_result['params']
    )

model.fit(X_train_numpy, y_train_numpy)
y_pred = model.predict(X_test_numpy).reshape(-1)
accuracy = np.mean([y_pred == y_test_numpy])
print(f'Best model accuracy: {accuracy}')
print(f'Best model params: {grid_search_result["params"]}')

bestTest = 1.275161267
bestIteration = 99
0:	loss: 1.2751613	best: 1.2751613 (0)	total: 1.77s	remaining: 1m 23s
bestTest = 0.8885701421
bestIteration = 99
1:	loss: 0.8885701	best: 0.8885701 (1)	total: 2.83s	remaining: 1m 5s
bestTest = 0.7470448793
bestIteration = 99
2:	loss: 0.7470449	best: 0.7470449 (2)	total: 3.9s	remaining: 58.5s
bestTest = 1.27701927
bestIteration = 99
3:	loss: 1.2770193	best: 0.7470449 (2)	total: 5.04s	remaining: 55.5s
bestTest = 0.8924142322
bestIteration = 99
4:	loss: 0.8924142	best: 0.7470449 (2)	total: 6.15s	remaining: 52.9s
bestTest = 0.7557229354
bestIteration = 99
5:	loss: 0.7557229	best: 0.7470449 (2)	total: 7.26s	remaining: 50.9s
bestTest = 1.279301007
bestIteration = 99
6:	loss: 1.2793010	best: 0.7470449 (2)	total: 8.33s	remaining: 48.8s
bestTest = 0.8951488092
bestIteration = 99
7:	loss: 0.8951488	best: 0.7470449 (2)	total: 9.48s	remaining: 47.4s
bestTest = 0.7601091572
bestIteration = 99
8:	loss: 0.7601092	best: 0.7470449 (2)	total: 10.5s	remaining: 45

49:	learn: 0.4633050	total: 13.6s	remaining: 22m 25s
50:	learn: 0.4565206	total: 13.9s	remaining: 22m 27s
51:	learn: 0.4494734	total: 14.2s	remaining: 22m 26s
52:	learn: 0.4433503	total: 14.4s	remaining: 22m 25s
53:	learn: 0.4376298	total: 14.7s	remaining: 22m 23s
54:	learn: 0.4317535	total: 14.9s	remaining: 22m 22s
55:	learn: 0.4265504	total: 15.2s	remaining: 22m 20s
56:	learn: 0.4207614	total: 15.4s	remaining: 22m 18s
57:	learn: 0.4148789	total: 15.7s	remaining: 22m 20s
58:	learn: 0.4093094	total: 16s	remaining: 22m 18s
59:	learn: 0.4025725	total: 16.3s	remaining: 22m 18s
60:	learn: 0.3967609	total: 16.5s	remaining: 22m 17s
61:	learn: 0.3910215	total: 16.8s	remaining: 22m 16s
62:	learn: 0.3862592	total: 17s	remaining: 22m 15s
63:	learn: 0.3811626	total: 17.3s	remaining: 22m 14s
64:	learn: 0.3770393	total: 17.6s	remaining: 22m 13s
65:	learn: 0.3722167	total: 17.8s	remaining: 22m 12s
66:	learn: 0.3673699	total: 18.1s	remaining: 22m 10s
67:	learn: 0.3628410	total: 18.3s	remaining: 22m 7

203:	learn: 0.1033853	total: 51.4s	remaining: 20m 7s
204:	learn: 0.1025043	total: 51.6s	remaining: 20m 7s
205:	learn: 0.1019320	total: 51.8s	remaining: 20m 6s
206:	learn: 0.1011813	total: 52.1s	remaining: 20m 5s
207:	learn: 0.1004356	total: 52.3s	remaining: 20m 5s
208:	learn: 0.0995753	total: 52.6s	remaining: 20m 5s
209:	learn: 0.0988173	total: 52.8s	remaining: 20m 5s
210:	learn: 0.0980130	total: 53.1s	remaining: 20m 5s
211:	learn: 0.0975177	total: 53.3s	remaining: 20m 4s
212:	learn: 0.0969308	total: 53.5s	remaining: 20m 3s
213:	learn: 0.0963608	total: 53.8s	remaining: 20m 2s
214:	learn: 0.0957144	total: 54s	remaining: 20m 1s
215:	learn: 0.0950228	total: 54.2s	remaining: 20m
216:	learn: 0.0941724	total: 54.5s	remaining: 20m
217:	learn: 0.0934470	total: 54.7s	remaining: 20m
218:	learn: 0.0927738	total: 55s	remaining: 20m
219:	learn: 0.0920196	total: 55.2s	remaining: 19m 59s
220:	learn: 0.0914399	total: 55.4s	remaining: 19m 58s
221:	learn: 0.0907720	total: 55.7s	remaining: 19m 58s
222:	l

355:	learn: 0.0423870	total: 1m 26s	remaining: 18m 48s
356:	learn: 0.0421916	total: 1m 26s	remaining: 18m 47s
357:	learn: 0.0419598	total: 1m 26s	remaining: 18m 47s
358:	learn: 0.0416644	total: 1m 27s	remaining: 18m 47s
359:	learn: 0.0414427	total: 1m 27s	remaining: 18m 47s
360:	learn: 0.0413041	total: 1m 27s	remaining: 18m 47s
361:	learn: 0.0410491	total: 1m 27s	remaining: 18m 47s
362:	learn: 0.0408007	total: 1m 28s	remaining: 18m 47s
363:	learn: 0.0406207	total: 1m 28s	remaining: 18m 46s
364:	learn: 0.0404047	total: 1m 28s	remaining: 18m 46s
365:	learn: 0.0401868	total: 1m 28s	remaining: 18m 46s
366:	learn: 0.0400337	total: 1m 29s	remaining: 18m 45s
367:	learn: 0.0398393	total: 1m 29s	remaining: 18m 44s
368:	learn: 0.0396587	total: 1m 29s	remaining: 18m 44s
369:	learn: 0.0395021	total: 1m 29s	remaining: 18m 43s
370:	learn: 0.0393090	total: 1m 30s	remaining: 18m 43s
371:	learn: 0.0391332	total: 1m 30s	remaining: 18m 42s
372:	learn: 0.0389194	total: 1m 30s	remaining: 18m 42s
373:	learn

506:	learn: 0.0223224	total: 2m 1s	remaining: 17m 55s
507:	learn: 0.0222241	total: 2m 1s	remaining: 17m 55s
508:	learn: 0.0221747	total: 2m 1s	remaining: 17m 54s
509:	learn: 0.0221062	total: 2m 2s	remaining: 17m 54s
510:	learn: 0.0220401	total: 2m 2s	remaining: 17m 53s
511:	learn: 0.0219606	total: 2m 2s	remaining: 17m 53s
512:	learn: 0.0218510	total: 2m 2s	remaining: 17m 53s
513:	learn: 0.0217742	total: 2m 2s	remaining: 17m 53s
514:	learn: 0.0216912	total: 2m 3s	remaining: 17m 52s
515:	learn: 0.0216254	total: 2m 3s	remaining: 17m 52s
516:	learn: 0.0215244	total: 2m 3s	remaining: 17m 52s
517:	learn: 0.0214800	total: 2m 3s	remaining: 17m 51s
518:	learn: 0.0213999	total: 2m 4s	remaining: 17m 51s
519:	learn: 0.0213095	total: 2m 4s	remaining: 17m 51s
520:	learn: 0.0212182	total: 2m 4s	remaining: 17m 50s
521:	learn: 0.0211429	total: 2m 4s	remaining: 17m 50s
522:	learn: 0.0210749	total: 2m 5s	remaining: 17m 50s
523:	learn: 0.0210007	total: 2m 5s	remaining: 17m 49s
524:	learn: 0.0209300	total:

...

4373:	learn: 0.0006583	total: 16m 18s	remaining: 2m 20s
4374:	learn: 0.0006581	total: 16m 19s	remaining: 2m 19s
4375:	learn: 0.0006579	total: 16m 19s	remaining: 2m 19s
4376:	learn: 0.0006577	total: 16m 19s	remaining: 2m 19s
4377:	learn: 0.0006575	total: 16m 19s	remaining: 2m 19s
4378:	learn: 0.0006573	total: 16m 19s	remaining: 2m 18s
4379:	learn: 0.0006571	total: 16m 20s	remaining: 2m 18s
4380:	learn: 0.0006570	total: 16m 20s	remaining: 2m 18s
4381:	learn: 0.0006568	total: 16m 20s	remaining: 2m 18s
4382:	learn: 0.0006566	total: 16m 20s	remaining: 2m 18s
4383:	learn: 0.0006564	total: 16m 20s	remaining: 2m 17s
4384:	learn: 0.0006561	total: 16m 21s	remaining: 2m 17s
4385:	learn: 0.0006559	total: 16m 21s	remaining: 2m 17s
4386:	learn: 0.0006558	total: 16m 21s	remaining: 2m 17s
4387:	learn: 0.0006556	total: 16m 21s	remaining: 2m 16s
4388:	learn: 0.0006554	total: 16m 22s	remaining: 2m 16s
4389:	learn: 0.0006552	total: 16m 22s	remaining: 2m 16s
4390:	learn: 0.0006550	total: 16m 22s	remaining:

4521:	learn: 0.0006299	total: 16m 50s	remaining: 1m 46s
4522:	learn: 0.0006297	total: 16m 51s	remaining: 1m 46s
4523:	learn: 0.0006296	total: 16m 51s	remaining: 1m 46s
4524:	learn: 0.0006294	total: 16m 51s	remaining: 1m 46s
4525:	learn: 0.0006292	total: 16m 51s	remaining: 1m 45s
4526:	learn: 0.0006290	total: 16m 51s	remaining: 1m 45s
4527:	learn: 0.0006288	total: 16m 52s	remaining: 1m 45s
4528:	learn: 0.0006287	total: 16m 52s	remaining: 1m 45s
4529:	learn: 0.0006285	total: 16m 52s	remaining: 1m 45s
4530:	learn: 0.0006283	total: 16m 52s	remaining: 1m 44s
4531:	learn: 0.0006281	total: 16m 53s	remaining: 1m 44s
4532:	learn: 0.0006279	total: 16m 53s	remaining: 1m 44s
4533:	learn: 0.0006277	total: 16m 53s	remaining: 1m 44s
4534:	learn: 0.0006275	total: 16m 53s	remaining: 1m 43s
4535:	learn: 0.0006274	total: 16m 53s	remaining: 1m 43s
4536:	learn: 0.0006272	total: 16m 54s	remaining: 1m 43s
4537:	learn: 0.0006270	total: 16m 54s	remaining: 1m 43s
4538:	learn: 0.0006268	total: 16m 54s	remaining:

4669:	learn: 0.0006040	total: 17m 22s	remaining: 1m 13s
4670:	learn: 0.0006039	total: 17m 23s	remaining: 1m 13s
4671:	learn: 0.0006037	total: 17m 23s	remaining: 1m 13s
4672:	learn: 0.0006035	total: 17m 23s	remaining: 1m 13s
4673:	learn: 0.0006034	total: 17m 23s	remaining: 1m 12s
4674:	learn: 0.0006032	total: 17m 23s	remaining: 1m 12s
4675:	learn: 0.0006030	total: 17m 24s	remaining: 1m 12s
4676:	learn: 0.0006029	total: 17m 24s	remaining: 1m 12s
4677:	learn: 0.0006027	total: 17m 24s	remaining: 1m 11s
4678:	learn: 0.0006026	total: 17m 24s	remaining: 1m 11s
4679:	learn: 0.0006024	total: 17m 24s	remaining: 1m 11s
4680:	learn: 0.0006022	total: 17m 25s	remaining: 1m 11s
4681:	learn: 0.0006021	total: 17m 25s	remaining: 1m 10s
4682:	learn: 0.0006019	total: 17m 25s	remaining: 1m 10s
4683:	learn: 0.0006017	total: 17m 25s	remaining: 1m 10s
4684:	learn: 0.0006016	total: 17m 25s	remaining: 1m 10s
4685:	learn: 0.0006014	total: 17m 26s	remaining: 1m 10s
4686:	learn: 0.0006012	total: 17m 26s	remaining:

4820:	learn: 0.0005795	total: 17m 54s	remaining: 39.9s
4821:	learn: 0.0005793	total: 17m 54s	remaining: 39.7s
4822:	learn: 0.0005792	total: 17m 55s	remaining: 39.5s
4823:	learn: 0.0005790	total: 17m 55s	remaining: 39.2s
4824:	learn: 0.0005788	total: 17m 55s	remaining: 39s
4825:	learn: 0.0005787	total: 17m 55s	remaining: 38.8s
4826:	learn: 0.0005785	total: 17m 56s	remaining: 38.6s
4827:	learn: 0.0005784	total: 17m 56s	remaining: 38.3s
4828:	learn: 0.0005782	total: 17m 56s	remaining: 38.1s
4829:	learn: 0.0005781	total: 17m 56s	remaining: 37.9s
4830:	learn: 0.0005779	total: 17m 56s	remaining: 37.7s
4831:	learn: 0.0005777	total: 17m 57s	remaining: 37.4s
4832:	learn: 0.0005776	total: 17m 57s	remaining: 37.2s
4833:	learn: 0.0005775	total: 17m 57s	remaining: 37s
4834:	learn: 0.0005773	total: 17m 57s	remaining: 36.8s
4835:	learn: 0.0005772	total: 17m 58s	remaining: 36.6s
4836:	learn: 0.0005770	total: 17m 58s	remaining: 36.3s
4837:	learn: 0.0005768	total: 17m 58s	remaining: 36.1s
4838:	learn: 0

4971:	learn: 0.0005568	total: 18m 27s	remaining: 6.24s
4972:	learn: 0.0005566	total: 18m 27s	remaining: 6.01s
4973:	learn: 0.0005565	total: 18m 27s	remaining: 5.79s
4974:	learn: 0.0005563	total: 18m 27s	remaining: 5.57s
4975:	learn: 0.0005562	total: 18m 28s	remaining: 5.34s
4976:	learn: 0.0005561	total: 18m 28s	remaining: 5.12s
4977:	learn: 0.0005559	total: 18m 28s	remaining: 4.9s
4978:	learn: 0.0005558	total: 18m 28s	remaining: 4.68s
4979:	learn: 0.0005557	total: 18m 28s	remaining: 4.45s
4980:	learn: 0.0005555	total: 18m 29s	remaining: 4.23s
4981:	learn: 0.0005554	total: 18m 29s	remaining: 4.01s
4982:	learn: 0.0005553	total: 18m 29s	remaining: 3.79s
4983:	learn: 0.0005551	total: 18m 29s	remaining: 3.56s
4984:	learn: 0.0005550	total: 18m 29s	remaining: 3.34s
4985:	learn: 0.0005548	total: 18m 30s	remaining: 3.12s
4986:	learn: 0.0005547	total: 18m 30s	remaining: 2.89s
4987:	learn: 0.0005546	total: 18m 30s	remaining: 2.67s
4988:	learn: 0.0005544	total: 18m 30s	remaining: 2.45s
4989:	learn

## Обучение на фичах других EfficientNet'ов

### EfficientNet B5

In [12]:
imagefolder = "C:/Users/User/images/456_white/"

In [13]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Using device:", device)

model = timm.create_model(
    'efficientnet_b5.sw_in12k_ft_in1k',
    pretrained=True,
    num_classes=0,       # => no classifier head
    global_pool='avg',   # => global average pooling
    features_only=False  # => return a single (B, 1280) output
)
model = model.to(device)
model.eval()

data_config = timm.data.resolve_data_config({}, model=model)
transform = timm.data.create_transform(**data_config, is_training=False)

def extract_features(images):
    """Run images through the model and return a (batch_size, 1280) tensor."""
    with torch.no_grad():
        # For features_only=False, the model returns a single tensor of shape (B, 1280)
        outputs = model(images)  # shape: (B, 1280)
        # No need to index with [-1] or do any flattening, it is already (B, 1280).
        return outputs

# Example dataset
dataset = ImageFolder(imagefolder, transform=transform)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

# Check feature dimension
sample_images, _ = next(iter(dataloader))
sample_images = sample_images.to(device)
sample_features = extract_features(sample_images)
feature_size = sample_features.size(1)
print(f"Sample feature size: {sample_features.shape}")  # Should be (B, 1280)

# Prepare storage for all features/labels
num_samples = len(dataset)
X = torch.zeros(num_samples, feature_size)
y = torch.zeros(num_samples, dtype=torch.long)

start_idx = 0
for images, labels in tqdm(dataloader):
    images = images.to(device)
    labels = labels.to(device)

    features = extract_features(images)  # shape: (batch_size, 1280)
    batch_size = features.size(0)

    # Move features and labels to CPU and store them
    X[start_idx:start_idx+batch_size] = features.cpu()
    y[start_idx:start_idx+batch_size] = labels.cpu()

    start_idx += batch_size

train_test_ratio = 0.8
train_size = round(train_test_ratio * len(X))
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]
print(f"Train size is {train_size}")
print(X_train.shape, X_test.shape)
print(y_train.shape, y_test.shape)

Using device: cuda
Sample feature size: torch.Size([32, 2048])


100%|██████████████████████████████████████████████████████████████████████████████| 1039/1039 [07:10<00:00,  2.41it/s]

Train size is 26586
torch.Size([26586, 2048]) torch.Size([6646, 2048])
torch.Size([26586]) torch.Size([6646])


In [14]:
# Экспорт фичей
X_train_numpy, X_test_numpy = X_train.numpy(), X_test.numpy()
y_train_numpy, y_test_numpy = y_train.numpy(), y_test.numpy()

np.savetxt("X_train_V1_B5_456.csv", X_train_numpy, delimiter=",")
np.savetxt("X_test_V1_B5_456.csv", X_test_numpy, delimiter=",")
np.savetxt("y_train_V1_B5_456.csv", y_train_numpy, delimiter=",")
np.savetxt("y_test_V1_B5_456.csv", y_test_numpy, delimiter=",")

In [5]:
# Импорт фичей
X_train_numpy = np.genfromtxt("X_train_V1_B5_456.csv", delimiter=",")
X_test_numpy = np.genfromtxt("X_test_V1_B5_456.csv", delimiter=",")
y_train_numpy = np.genfromtxt("y_train_V1_B5_456.csv", delimiter=",")
y_test_numpy = np.genfromtxt("y_test_V1_B5_456.csv", delimiter=",")

In [6]:
print(X_train_numpy.shape)
print(X_test_numpy.shape)
print(y_train_numpy.shape)
print(y_test_numpy.shape)

(26586, 2048)
(6646, 2048)
(26586,)
(6646,)


In [7]:
# Здесь эскпериментируем с модельками
model = CatBoostClassifier(
    iterations=100, 
    loss_function='MultiClass', 
    verbose=False,
    task_type='GPU',
    devices='0'
    )

grid = {'learning_rate': [0.01, 0.05, 0.1],
        'depth': [4, 6, 8, 10],
        'l2_leaf_reg': [1, 3, 5, 7]
       }

grid_search_result = model.grid_search(grid,
                                       X=X_train_numpy,
                                       y=y_train_numpy,
                                       refit=False
                                      )

model = CatBoostClassifier(
    iterations=5000, 
    loss_function='MultiClass', 
    verbose=True,
    task_type='GPU',
    devices='0',
    **grid_search_result['params']
    )

model.fit(X_train_numpy, y_train_numpy)
y_pred = model.predict(X_test_numpy).reshape(-1)
accuracy = np.mean([y_pred == y_test_numpy])
print(f'Best model accuracy: {accuracy}')
print(f'Best model params: {grid_search_result["params"]}')

bestTest = 1.2819232
bestIteration = 99
0:	loss: 1.2819232	best: 1.2819232 (0)	total: 2.25s	remaining: 1m 45s
bestTest = 0.9109763568
bestIteration = 99
1:	loss: 0.9109764	best: 0.9109764 (1)	total: 3.81s	remaining: 1m 27s
bestTest = 0.7808383857
bestIteration = 99
2:	loss: 0.7808384	best: 0.7808384 (2)	total: 5.4s	remaining: 1m 20s
bestTest = 1.284444854
bestIteration = 99
3:	loss: 1.2844449	best: 0.7808384 (2)	total: 6.94s	remaining: 1m 16s
bestTest = 0.9135777996
bestIteration = 99
4:	loss: 0.9135778	best: 0.7808384 (2)	total: 8.63s	remaining: 1m 14s
bestTest = 0.7904196428
bestIteration = 99
5:	loss: 0.7904196	best: 0.7808384 (2)	total: 10.2s	remaining: 1m 11s
bestTest = 1.287304945
bestIteration = 99
6:	loss: 1.2873049	best: 0.7808384 (2)	total: 11.7s	remaining: 1m 8s
bestTest = 0.9190991055
bestIteration = 99
7:	loss: 0.9190991	best: 0.7808384 (2)	total: 13.2s	remaining: 1m 6s
bestTest = 0.7921675574
bestIteration = 99
8:	loss: 0.7921676	best: 0.7808384 (2)	total: 14.8s	remaining

49:	learn: 0.4437829	total: 20.6s	remaining: 33m 59s
50:	learn: 0.4369465	total: 21s	remaining: 33m 58s
51:	learn: 0.4314195	total: 21.4s	remaining: 33m 52s
52:	learn: 0.4245505	total: 21.8s	remaining: 33m 53s
53:	learn: 0.4184049	total: 22.2s	remaining: 33m 52s
54:	learn: 0.4112939	total: 22.6s	remaining: 33m 52s
55:	learn: 0.4048111	total: 23s	remaining: 33m 52s
56:	learn: 0.3981965	total: 23.4s	remaining: 33m 53s
57:	learn: 0.3930758	total: 23.8s	remaining: 33m 50s
58:	learn: 0.3877678	total: 24.2s	remaining: 33m 48s
59:	learn: 0.3823597	total: 24.6s	remaining: 33m 47s
60:	learn: 0.3782627	total: 25s	remaining: 33m 41s
61:	learn: 0.3728261	total: 25.3s	remaining: 33m 37s
62:	learn: 0.3683099	total: 25.7s	remaining: 33m 36s
63:	learn: 0.3641664	total: 26.1s	remaining: 33m 34s
64:	learn: 0.3594520	total: 26.5s	remaining: 33m 31s
65:	learn: 0.3540218	total: 26.9s	remaining: 33m 31s
66:	learn: 0.3498138	total: 27.3s	remaining: 33m 31s
67:	learn: 0.3454413	total: 27.7s	remaining: 33m 26s

203:	learn: 0.1039544	total: 1m 16s	remaining: 29m 58s
204:	learn: 0.1032966	total: 1m 16s	remaining: 29m 57s
205:	learn: 0.1023569	total: 1m 17s	remaining: 29m 58s
206:	learn: 0.1015998	total: 1m 17s	remaining: 29m 57s
207:	learn: 0.1007690	total: 1m 18s	remaining: 29m 57s
208:	learn: 0.1001098	total: 1m 18s	remaining: 29m 56s
209:	learn: 0.0995793	total: 1m 18s	remaining: 29m 55s
210:	learn: 0.0989806	total: 1m 19s	remaining: 29m 53s
211:	learn: 0.0984526	total: 1m 19s	remaining: 29m 52s
212:	learn: 0.0980285	total: 1m 19s	remaining: 29m 50s
213:	learn: 0.0976088	total: 1m 19s	remaining: 29m 48s
214:	learn: 0.0970702	total: 1m 20s	remaining: 29m 46s
215:	learn: 0.0964001	total: 1m 20s	remaining: 29m 45s
216:	learn: 0.0960234	total: 1m 20s	remaining: 29m 43s
217:	learn: 0.0952631	total: 1m 21s	remaining: 29m 43s
218:	learn: 0.0946006	total: 1m 21s	remaining: 29m 42s
219:	learn: 0.0939286	total: 1m 22s	remaining: 29m 42s
220:	learn: 0.0933564	total: 1m 22s	remaining: 29m 41s
221:	learn

354:	learn: 0.0448694	total: 2m 8s	remaining: 27m 58s
355:	learn: 0.0447056	total: 2m 8s	remaining: 27m 57s
356:	learn: 0.0444661	total: 2m 8s	remaining: 27m 57s
357:	learn: 0.0442698	total: 2m 9s	remaining: 27m 56s
358:	learn: 0.0441060	total: 2m 9s	remaining: 27m 55s
359:	learn: 0.0439311	total: 2m 9s	remaining: 27m 54s
360:	learn: 0.0437526	total: 2m 10s	remaining: 27m 54s
361:	learn: 0.0435514	total: 2m 10s	remaining: 27m 53s
362:	learn: 0.0433444	total: 2m 10s	remaining: 27m 52s
363:	learn: 0.0431427	total: 2m 11s	remaining: 27m 52s
364:	learn: 0.0428989	total: 2m 11s	remaining: 27m 52s
365:	learn: 0.0427473	total: 2m 12s	remaining: 27m 51s
366:	learn: 0.0426539	total: 2m 12s	remaining: 27m 49s
367:	learn: 0.0424717	total: 2m 12s	remaining: 27m 48s
368:	learn: 0.0423462	total: 2m 12s	remaining: 27m 47s
369:	learn: 0.0421300	total: 2m 13s	remaining: 27m 47s
370:	learn: 0.0419835	total: 2m 13s	remaining: 27m 46s
371:	learn: 0.0418455	total: 2m 13s	remaining: 27m 45s
372:	learn: 0.04

504:	learn: 0.0239082	total: 2m 59s	remaining: 26m 41s
505:	learn: 0.0238017	total: 3m	remaining: 26m 40s
506:	learn: 0.0237140	total: 3m	remaining: 26m 40s
507:	learn: 0.0236161	total: 3m	remaining: 26m 39s
508:	learn: 0.0235298	total: 3m 1s	remaining: 26m 39s
509:	learn: 0.0234263	total: 3m 1s	remaining: 26m 39s
510:	learn: 0.0233385	total: 3m 1s	remaining: 26m 38s
511:	learn: 0.0232643	total: 3m 2s	remaining: 26m 37s
512:	learn: 0.0231688	total: 3m 2s	remaining: 26m 37s
513:	learn: 0.0230943	total: 3m 2s	remaining: 26m 36s
514:	learn: 0.0230100	total: 3m 3s	remaining: 26m 36s
515:	learn: 0.0229341	total: 3m 3s	remaining: 26m 35s
516:	learn: 0.0228531	total: 3m 3s	remaining: 26m 35s
517:	learn: 0.0227708	total: 3m 4s	remaining: 26m 35s
518:	learn: 0.0227070	total: 3m 4s	remaining: 26m 34s
519:	learn: 0.0226149	total: 3m 5s	remaining: 26m 33s
520:	learn: 0.0225162	total: 3m 5s	remaining: 26m 33s
521:	learn: 0.0224272	total: 3m 5s	remaining: 26m 33s
522:	learn: 0.0223651	total: 3m 6s	r

...

4335:	learn: 0.0006711	total: 24m 19s	remaining: 3m 43s
4336:	learn: 0.0006710	total: 24m 19s	remaining: 3m 43s
4337:	learn: 0.0006708	total: 24m 19s	remaining: 3m 42s
4338:	learn: 0.0006705	total: 24m 20s	remaining: 3m 42s
4339:	learn: 0.0006703	total: 24m 20s	remaining: 3m 42s
4340:	learn: 0.0006701	total: 24m 20s	remaining: 3m 41s
4341:	learn: 0.0006699	total: 24m 21s	remaining: 3m 41s
4342:	learn: 0.0006697	total: 24m 21s	remaining: 3m 41s
4343:	learn: 0.0006695	total: 24m 21s	remaining: 3m 40s
4344:	learn: 0.0006692	total: 24m 21s	remaining: 3m 40s
4345:	learn: 0.0006690	total: 24m 22s	remaining: 3m 40s
4346:	learn: 0.0006688	total: 24m 22s	remaining: 3m 39s
4347:	learn: 0.0006686	total: 24m 23s	remaining: 3m 39s
4348:	learn: 0.0006685	total: 24m 23s	remaining: 3m 39s
4349:	learn: 0.0006683	total: 24m 23s	remaining: 3m 38s
4350:	learn: 0.0006680	total: 24m 24s	remaining: 3m 38s
4351:	learn: 0.0006678	total: 24m 24s	remaining: 3m 38s
4352:	learn: 0.0006676	total: 24m 24s	remaining:

4483:	learn: 0.0006424	total: 25m 8s	remaining: 2m 53s
4484:	learn: 0.0006422	total: 25m 8s	remaining: 2m 53s
4485:	learn: 0.0006420	total: 25m 8s	remaining: 2m 52s
4486:	learn: 0.0006418	total: 25m 9s	remaining: 2m 52s
4487:	learn: 0.0006416	total: 25m 9s	remaining: 2m 52s
4488:	learn: 0.0006414	total: 25m 9s	remaining: 2m 51s
4489:	learn: 0.0006413	total: 25m 10s	remaining: 2m 51s
4490:	learn: 0.0006411	total: 25m 10s	remaining: 2m 51s
4491:	learn: 0.0006409	total: 25m 10s	remaining: 2m 50s
4492:	learn: 0.0006407	total: 25m 11s	remaining: 2m 50s
4493:	learn: 0.0006406	total: 25m 11s	remaining: 2m 50s
4494:	learn: 0.0006404	total: 25m 11s	remaining: 2m 49s
4495:	learn: 0.0006401	total: 25m 12s	remaining: 2m 49s
4496:	learn: 0.0006399	total: 25m 12s	remaining: 2m 49s
4497:	learn: 0.0006397	total: 25m 12s	remaining: 2m 48s
4498:	learn: 0.0006395	total: 25m 13s	remaining: 2m 48s
4499:	learn: 0.0006393	total: 25m 13s	remaining: 2m 48s
4500:	learn: 0.0006391	total: 25m 13s	remaining: 2m 47

4630:	learn: 0.0006157	total: 25m 56s	remaining: 2m 4s
4631:	learn: 0.0006155	total: 25m 56s	remaining: 2m 3s
4632:	learn: 0.0006153	total: 25m 56s	remaining: 2m 3s
4633:	learn: 0.0006152	total: 25m 57s	remaining: 2m 2s
4634:	learn: 0.0006150	total: 25m 57s	remaining: 2m 2s
4635:	learn: 0.0006148	total: 25m 57s	remaining: 2m 2s
4636:	learn: 0.0006147	total: 25m 58s	remaining: 2m 1s
4637:	learn: 0.0006145	total: 25m 58s	remaining: 2m 1s
4638:	learn: 0.0006143	total: 25m 58s	remaining: 2m 1s
4639:	learn: 0.0006141	total: 25m 59s	remaining: 2m
4640:	learn: 0.0006140	total: 25m 59s	remaining: 2m
4641:	learn: 0.0006138	total: 25m 59s	remaining: 2m
4642:	learn: 0.0006136	total: 26m	remaining: 1m 59s
4643:	learn: 0.0006135	total: 26m	remaining: 1m 59s
4644:	learn: 0.0006133	total: 26m	remaining: 1m 59s
4645:	learn: 0.0006131	total: 26m 1s	remaining: 1m 58s
4646:	learn: 0.0006129	total: 26m 1s	remaining: 1m 58s
4647:	learn: 0.0006128	total: 26m 1s	remaining: 1m 58s
4648:	learn: 0.0006126	total

4778:	learn: 0.0005912	total: 26m 44s	remaining: 1m 14s
4779:	learn: 0.0005911	total: 26m 44s	remaining: 1m 13s
4780:	learn: 0.0005909	total: 26m 44s	remaining: 1m 13s
4781:	learn: 0.0005908	total: 26m 45s	remaining: 1m 13s
4782:	learn: 0.0005906	total: 26m 45s	remaining: 1m 12s
4783:	learn: 0.0005904	total: 26m 45s	remaining: 1m 12s
4784:	learn: 0.0005903	total: 26m 46s	remaining: 1m 12s
4785:	learn: 0.0005901	total: 26m 46s	remaining: 1m 11s
4786:	learn: 0.0005899	total: 26m 46s	remaining: 1m 11s
4787:	learn: 0.0005898	total: 26m 47s	remaining: 1m 11s
4788:	learn: 0.0005897	total: 26m 47s	remaining: 1m 10s
4789:	learn: 0.0005895	total: 26m 47s	remaining: 1m 10s
4790:	learn: 0.0005893	total: 26m 48s	remaining: 1m 10s
4791:	learn: 0.0005892	total: 26m 48s	remaining: 1m 9s
4792:	learn: 0.0005890	total: 26m 48s	remaining: 1m 9s
4793:	learn: 0.0005889	total: 26m 49s	remaining: 1m 9s
4794:	learn: 0.0005887	total: 26m 49s	remaining: 1m 8s
4795:	learn: 0.0005886	total: 26m 49s	remaining: 1m 

4929:	learn: 0.0005672	total: 27m 34s	remaining: 23.5s
4930:	learn: 0.0005671	total: 27m 34s	remaining: 23.2s
4931:	learn: 0.0005669	total: 27m 34s	remaining: 22.8s
4932:	learn: 0.0005668	total: 27m 35s	remaining: 22.5s
4933:	learn: 0.0005666	total: 27m 35s	remaining: 22.1s
4934:	learn: 0.0005665	total: 27m 35s	remaining: 21.8s
4935:	learn: 0.0005663	total: 27m 36s	remaining: 21.5s
4936:	learn: 0.0005661	total: 27m 36s	remaining: 21.1s
4937:	learn: 0.0005660	total: 27m 36s	remaining: 20.8s
4938:	learn: 0.0005659	total: 27m 37s	remaining: 20.5s
4939:	learn: 0.0005657	total: 27m 37s	remaining: 20.1s
4940:	learn: 0.0005656	total: 27m 37s	remaining: 19.8s
4941:	learn: 0.0005654	total: 27m 38s	remaining: 19.5s
4942:	learn: 0.0005653	total: 27m 38s	remaining: 19.1s
4943:	learn: 0.0005651	total: 27m 38s	remaining: 18.8s
4944:	learn: 0.0005650	total: 27m 39s	remaining: 18.5s
4945:	learn: 0.0005648	total: 27m 39s	remaining: 18.1s
4946:	learn: 0.0005646	total: 27m 40s	remaining: 17.8s
4947:	lear

### EfficientNet-v2 M

In [17]:
imagefolder = "C:/Users/User/images/480_white/"

In [18]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Using device:", device)

model = timm.create_model(
    'tf_efficientnetv2_m.in21k_ft_in1k',
    pretrained=True,
    num_classes=0,       # => no classifier head
    global_pool='avg',   # => global average pooling
    features_only=False  # => return a single (B, 1280) output
)
model = model.to(device)
model.eval()

data_config = timm.data.resolve_data_config({}, model=model)
transform = timm.data.create_transform(**data_config, is_training=False)

def extract_features(images):
    """Run images through the model and return a (batch_size, 1280) tensor."""
    with torch.no_grad():
        # For features_only=False, the model returns a single tensor of shape (B, 1280)
        outputs = model(images)  # shape: (B, 1280)
        # No need to index with [-1] or do any flattening, it is already (B, 1280).
        return outputs

# Example dataset
dataset = ImageFolder(imagefolder, transform=transform)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

# Check feature dimension
sample_images, _ = next(iter(dataloader))
sample_images = sample_images.to(device)
sample_features = extract_features(sample_images)
feature_size = sample_features.size(1)
print(f"Sample feature size: {sample_features.shape}")  # Should be (B, 1280)

# Prepare storage for all features/labels
num_samples = len(dataset)
X = torch.zeros(num_samples, feature_size)
y = torch.zeros(num_samples, dtype=torch.long)

start_idx = 0
for images, labels in tqdm(dataloader):
    images = images.to(device)
    labels = labels.to(device)

    features = extract_features(images)  # shape: (batch_size, 1280)
    batch_size = features.size(0)

    # Move features and labels to CPU and store them
    X[start_idx:start_idx+batch_size] = features.cpu()
    y[start_idx:start_idx+batch_size] = labels.cpu()

    start_idx += batch_size

train_test_ratio = 0.8
train_size = round(train_test_ratio * len(X))
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]
print(f"Train size is {train_size}")
print(X_train.shape, X_test.shape)
print(y_train.shape, y_test.shape)

Using device: cuda
Sample feature size: torch.Size([32, 1280])


100%|██████████████████████████████████████████████████████████████████████████████| 1039/1039 [06:50<00:00,  2.53it/s]

Train size is 26586
torch.Size([26586, 1280]) torch.Size([6646, 1280])
torch.Size([26586]) torch.Size([6646])


In [19]:
# Экспорт фичей
X_train_numpy, X_test_numpy = X_train.numpy(), X_test.numpy()
y_train_numpy, y_test_numpy = y_train.numpy(), y_test.numpy()

np.savetxt("X_train_V2_M_480.csv", X_train_numpy, delimiter=",")
np.savetxt("X_test_V2_M_480.csv", X_test_numpy, delimiter=",")
np.savetxt("y_train_V2_M_480.csv", y_train_numpy, delimiter=",")
np.savetxt("y_test_V2_M_480.csv", y_test_numpy, delimiter=",")

In [8]:
# Импорт фичей
X_train_numpy = np.genfromtxt("X_train_V2_M_480.csv", delimiter=",")
X_test_numpy = np.genfromtxt("X_test_V2_M_480.csv", delimiter=",")
y_train_numpy = np.genfromtxt("y_train_V2_M_480.csv", delimiter=",")
y_test_numpy = np.genfromtxt("y_test_V2_M_480.csv", delimiter=",")

In [9]:
print(X_train_numpy.shape)
print(X_test_numpy.shape)
print(y_train_numpy.shape)
print(y_test_numpy.shape)

(26586, 1280)
(6646, 1280)
(26586,)
(6646,)


In [10]:
# Здесь эскпериментируем с модельками
model = CatBoostClassifier(
    iterations=100, 
    loss_function='MultiClass', 
    verbose=False,
    task_type='GPU',
    devices='0'
    )

grid = {'learning_rate': [0.01, 0.05, 0.1],
        'depth': [4, 6, 8, 10],
        'l2_leaf_reg': [1, 3, 5, 7]
       }

grid_search_result = model.grid_search(grid,
                                       X=X_train_numpy,
                                       y=y_train_numpy,
                                       refit=False
                                      )

model = CatBoostClassifier(
    iterations=5000, 
    loss_function='MultiClass', 
    verbose=True,
    task_type='GPU',
    devices='0',
    **grid_search_result['params']
    )

model.fit(X_train_numpy, y_train_numpy)
y_pred = model.predict(X_test_numpy).reshape(-1)
accuracy = np.mean([y_pred == y_test_numpy])
print(f'Best model accuracy: {accuracy}')
print(f'Best model params: {grid_search_result["params"]}')

bestTest = 1.314255168
bestIteration = 99
0:	loss: 1.3142552	best: 1.3142552 (0)	total: 1.51s	remaining: 1m 10s
bestTest = 0.9133370562
bestIteration = 99
1:	loss: 0.9133371	best: 0.9133371 (1)	total: 2.65s	remaining: 1m 1s
bestTest = 0.7617249935
bestIteration = 99
2:	loss: 0.7617250	best: 0.7617250 (2)	total: 3.73s	remaining: 56s
bestTest = 1.317082847
bestIteration = 99
3:	loss: 1.3170828	best: 0.7617250 (2)	total: 4.81s	remaining: 52.9s
bestTest = 0.9179471731
bestIteration = 99
4:	loss: 0.9179472	best: 0.7617250 (2)	total: 5.93s	remaining: 51s
bestTest = 0.7705638207
bestIteration = 99
5:	loss: 0.7705638	best: 0.7617250 (2)	total: 6.98s	remaining: 48.9s
bestTest = 1.31943262
bestIteration = 99
6:	loss: 1.3194326	best: 0.7617250 (2)	total: 8.14s	remaining: 47.7s
bestTest = 0.9214521841
bestIteration = 99
7:	loss: 0.9214522	best: 0.7617250 (2)	total: 9.22s	remaining: 46.1s
bestTest = 0.7749820407
bestIteration = 99
8:	loss: 0.7749820	best: 0.7617250 (2)	total: 10.3s	remaining: 44.7s

49:	learn: 0.4504350	total: 13.1s	remaining: 21m 32s
50:	learn: 0.4440465	total: 13.3s	remaining: 21m 31s
51:	learn: 0.4357660	total: 13.6s	remaining: 21m 32s
52:	learn: 0.4288356	total: 13.9s	remaining: 21m 33s
53:	learn: 0.4220992	total: 14.1s	remaining: 21m 33s
54:	learn: 0.4160675	total: 14.4s	remaining: 21m 33s
55:	learn: 0.4094032	total: 14.7s	remaining: 21m 34s
56:	learn: 0.4029901	total: 14.9s	remaining: 21m 34s
57:	learn: 0.3969488	total: 15.2s	remaining: 21m 34s
58:	learn: 0.3919473	total: 15.4s	remaining: 21m 32s
59:	learn: 0.3867318	total: 15.7s	remaining: 21m 32s
60:	learn: 0.3803870	total: 16s	remaining: 21m 32s
61:	learn: 0.3759072	total: 16.2s	remaining: 21m 28s
62:	learn: 0.3709013	total: 16.4s	remaining: 21m 26s
63:	learn: 0.3656391	total: 16.7s	remaining: 21m 24s
64:	learn: 0.3615141	total: 16.9s	remaining: 21m 21s
65:	learn: 0.3563292	total: 17.2s	remaining: 21m 22s
66:	learn: 0.3519716	total: 17.4s	remaining: 21m 21s
67:	learn: 0.3474984	total: 17.6s	remaining: 21m

204:	learn: 0.0936337	total: 51.1s	remaining: 19m 54s
205:	learn: 0.0930009	total: 51.3s	remaining: 19m 53s
206:	learn: 0.0922262	total: 51.5s	remaining: 19m 53s
207:	learn: 0.0916929	total: 51.8s	remaining: 19m 52s
208:	learn: 0.0910747	total: 52s	remaining: 19m 52s
209:	learn: 0.0903865	total: 52.2s	remaining: 19m 51s
210:	learn: 0.0898283	total: 52.4s	remaining: 19m 50s
211:	learn: 0.0890868	total: 52.7s	remaining: 19m 50s
212:	learn: 0.0884984	total: 53s	remaining: 19m 50s
213:	learn: 0.0876410	total: 53.2s	remaining: 19m 50s
214:	learn: 0.0869647	total: 53.5s	remaining: 19m 50s
215:	learn: 0.0863854	total: 53.7s	remaining: 19m 49s
216:	learn: 0.0857288	total: 54s	remaining: 19m 49s
217:	learn: 0.0851372	total: 54.2s	remaining: 19m 49s
218:	learn: 0.0845935	total: 54.5s	remaining: 19m 48s
219:	learn: 0.0838496	total: 54.7s	remaining: 19m 48s
220:	learn: 0.0832242	total: 55s	remaining: 19m 48s
221:	learn: 0.0825851	total: 55.2s	remaining: 19m 48s
222:	learn: 0.0820343	total: 55.5s	r

356:	learn: 0.0379935	total: 1m 26s	remaining: 18m 47s
357:	learn: 0.0377774	total: 1m 26s	remaining: 18m 47s
358:	learn: 0.0375906	total: 1m 27s	remaining: 18m 46s
359:	learn: 0.0374152	total: 1m 27s	remaining: 18m 46s
360:	learn: 0.0371807	total: 1m 27s	remaining: 18m 46s
361:	learn: 0.0370367	total: 1m 27s	remaining: 18m 45s
362:	learn: 0.0368267	total: 1m 28s	remaining: 18m 45s
363:	learn: 0.0366656	total: 1m 28s	remaining: 18m 45s
364:	learn: 0.0364997	total: 1m 28s	remaining: 18m 44s
365:	learn: 0.0363514	total: 1m 28s	remaining: 18m 44s
366:	learn: 0.0361566	total: 1m 29s	remaining: 18m 44s
367:	learn: 0.0359477	total: 1m 29s	remaining: 18m 43s
368:	learn: 0.0357309	total: 1m 29s	remaining: 18m 44s
369:	learn: 0.0355310	total: 1m 29s	remaining: 18m 44s
370:	learn: 0.0353531	total: 1m 30s	remaining: 18m 43s
371:	learn: 0.0352136	total: 1m 30s	remaining: 18m 43s
372:	learn: 0.0350744	total: 1m 30s	remaining: 18m 42s
373:	learn: 0.0348820	total: 1m 30s	remaining: 18m 42s
374:	learn

506:	learn: 0.0198519	total: 2m 1s	remaining: 18m
507:	learn: 0.0197863	total: 2m 2s	remaining: 18m
508:	learn: 0.0197002	total: 2m 2s	remaining: 17m 59s
509:	learn: 0.0196211	total: 2m 2s	remaining: 17m 59s
510:	learn: 0.0195312	total: 2m 2s	remaining: 17m 59s
511:	learn: 0.0194478	total: 2m 3s	remaining: 17m 59s
512:	learn: 0.0193640	total: 2m 3s	remaining: 17m 59s
513:	learn: 0.0192938	total: 2m 3s	remaining: 17m 59s
514:	learn: 0.0192161	total: 2m 3s	remaining: 17m 59s
515:	learn: 0.0191453	total: 2m 4s	remaining: 17m 59s
516:	learn: 0.0190963	total: 2m 4s	remaining: 17m 58s
517:	learn: 0.0190285	total: 2m 4s	remaining: 17m 58s
518:	learn: 0.0189664	total: 2m 4s	remaining: 17m 57s
519:	learn: 0.0189057	total: 2m 5s	remaining: 17m 57s
520:	learn: 0.0188349	total: 2m 5s	remaining: 17m 56s
521:	learn: 0.0187669	total: 2m 5s	remaining: 17m 56s
522:	learn: 0.0187186	total: 2m 5s	remaining: 17m 55s
523:	learn: 0.0186526	total: 2m 5s	remaining: 17m 55s
524:	learn: 0.0185706	total: 2m 6s	r

...

4370:	learn: 0.0006235	total: 16m 30s	remaining: 2m 22s
4371:	learn: 0.0006233	total: 16m 30s	remaining: 2m 22s
4372:	learn: 0.0006232	total: 16m 30s	remaining: 2m 22s
4373:	learn: 0.0006229	total: 16m 30s	remaining: 2m 21s
4374:	learn: 0.0006227	total: 16m 31s	remaining: 2m 21s
4375:	learn: 0.0006225	total: 16m 31s	remaining: 2m 21s
4376:	learn: 0.0006224	total: 16m 31s	remaining: 2m 21s
4377:	learn: 0.0006222	total: 16m 31s	remaining: 2m 20s
4378:	learn: 0.0006220	total: 16m 31s	remaining: 2m 20s
4379:	learn: 0.0006218	total: 16m 32s	remaining: 2m 20s
4380:	learn: 0.0006217	total: 16m 32s	remaining: 2m 20s
4381:	learn: 0.0006215	total: 16m 32s	remaining: 2m 19s
4382:	learn: 0.0006213	total: 16m 32s	remaining: 2m 19s
4383:	learn: 0.0006211	total: 16m 32s	remaining: 2m 19s
4384:	learn: 0.0006210	total: 16m 33s	remaining: 2m 19s
4385:	learn: 0.0006208	total: 16m 33s	remaining: 2m 19s
4386:	learn: 0.0006206	total: 16m 33s	remaining: 2m 18s
4387:	learn: 0.0006204	total: 16m 33s	remaining:

4518:	learn: 0.0005969	total: 17m 1s	remaining: 1m 48s
4519:	learn: 0.0005968	total: 17m 2s	remaining: 1m 48s
4520:	learn: 0.0005966	total: 17m 2s	remaining: 1m 48s
4521:	learn: 0.0005964	total: 17m 2s	remaining: 1m 48s
4522:	learn: 0.0005963	total: 17m 2s	remaining: 1m 47s
4523:	learn: 0.0005961	total: 17m 2s	remaining: 1m 47s
4524:	learn: 0.0005960	total: 17m 3s	remaining: 1m 47s
4525:	learn: 0.0005958	total: 17m 3s	remaining: 1m 47s
4526:	learn: 0.0005957	total: 17m 3s	remaining: 1m 46s
4527:	learn: 0.0005955	total: 17m 3s	remaining: 1m 46s
4528:	learn: 0.0005953	total: 17m 3s	remaining: 1m 46s
4529:	learn: 0.0005952	total: 17m 4s	remaining: 1m 46s
4530:	learn: 0.0005950	total: 17m 4s	remaining: 1m 46s
4531:	learn: 0.0005948	total: 17m 4s	remaining: 1m 45s
4532:	learn: 0.0005947	total: 17m 4s	remaining: 1m 45s
4533:	learn: 0.0005945	total: 17m 5s	remaining: 1m 45s
4534:	learn: 0.0005944	total: 17m 5s	remaining: 1m 45s
4535:	learn: 0.0005942	total: 17m 5s	remaining: 1m 44s
4536:	lear

4665:	learn: 0.0005730	total: 17m 33s	remaining: 1m 15s
4666:	learn: 0.0005729	total: 17m 34s	remaining: 1m 15s
4667:	learn: 0.0005727	total: 17m 34s	remaining: 1m 14s
4668:	learn: 0.0005726	total: 17m 34s	remaining: 1m 14s
4669:	learn: 0.0005724	total: 17m 34s	remaining: 1m 14s
4670:	learn: 0.0005723	total: 17m 34s	remaining: 1m 14s
4671:	learn: 0.0005721	total: 17m 35s	remaining: 1m 14s
4672:	learn: 0.0005720	total: 17m 35s	remaining: 1m 13s
4673:	learn: 0.0005718	total: 17m 35s	remaining: 1m 13s
4674:	learn: 0.0005716	total: 17m 35s	remaining: 1m 13s
4675:	learn: 0.0005714	total: 17m 36s	remaining: 1m 13s
4676:	learn: 0.0005713	total: 17m 36s	remaining: 1m 12s
4677:	learn: 0.0005712	total: 17m 36s	remaining: 1m 12s
4678:	learn: 0.0005710	total: 17m 36s	remaining: 1m 12s
4679:	learn: 0.0005708	total: 17m 36s	remaining: 1m 12s
4680:	learn: 0.0005707	total: 17m 37s	remaining: 1m 12s
4681:	learn: 0.0005705	total: 17m 37s	remaining: 1m 11s
4682:	learn: 0.0005703	total: 17m 37s	remaining:

4815:	learn: 0.0005500	total: 18m 6s	remaining: 41.5s
4816:	learn: 0.0005498	total: 18m 6s	remaining: 41.3s
4817:	learn: 0.0005497	total: 18m 6s	remaining: 41s
4818:	learn: 0.0005495	total: 18m 6s	remaining: 40.8s
4819:	learn: 0.0005494	total: 18m 6s	remaining: 40.6s
4820:	learn: 0.0005492	total: 18m 7s	remaining: 40.4s
4821:	learn: 0.0005491	total: 18m 7s	remaining: 40.1s
4822:	learn: 0.0005489	total: 18m 7s	remaining: 39.9s
4823:	learn: 0.0005487	total: 18m 7s	remaining: 39.7s
4824:	learn: 0.0005485	total: 18m 7s	remaining: 39.5s
4825:	learn: 0.0005484	total: 18m 8s	remaining: 39.2s
4826:	learn: 0.0005482	total: 18m 8s	remaining: 39s
4827:	learn: 0.0005481	total: 18m 8s	remaining: 38.8s
4828:	learn: 0.0005479	total: 18m 8s	remaining: 38.6s
4829:	learn: 0.0005478	total: 18m 9s	remaining: 38.3s
4830:	learn: 0.0005477	total: 18m 9s	remaining: 38.1s
4831:	learn: 0.0005475	total: 18m 9s	remaining: 37.9s
4832:	learn: 0.0005474	total: 18m 9s	remaining: 37.7s
4833:	learn: 0.0005472	total: 18

4965:	learn: 0.0005295	total: 18m 38s	remaining: 7.66s
4966:	learn: 0.0005293	total: 18m 38s	remaining: 7.43s
4967:	learn: 0.0005292	total: 18m 39s	remaining: 7.21s
4968:	learn: 0.0005291	total: 18m 39s	remaining: 6.98s
4969:	learn: 0.0005290	total: 18m 39s	remaining: 6.76s
4970:	learn: 0.0005288	total: 18m 39s	remaining: 6.53s
4971:	learn: 0.0005287	total: 18m 39s	remaining: 6.31s
4972:	learn: 0.0005285	total: 18m 40s	remaining: 6.08s
4973:	learn: 0.0005284	total: 18m 40s	remaining: 5.86s
4974:	learn: 0.0005282	total: 18m 40s	remaining: 5.63s
4975:	learn: 0.0005281	total: 18m 40s	remaining: 5.41s
4976:	learn: 0.0005279	total: 18m 41s	remaining: 5.18s
4977:	learn: 0.0005278	total: 18m 41s	remaining: 4.96s
4978:	learn: 0.0005277	total: 18m 41s	remaining: 4.73s
4979:	learn: 0.0005275	total: 18m 41s	remaining: 4.5s
4980:	learn: 0.0005274	total: 18m 41s	remaining: 4.28s
4981:	learn: 0.0005273	total: 18m 42s	remaining: 4.05s
4982:	learn: 0.0005272	total: 18m 42s	remaining: 3.83s
4983:	learn

### EfficientNet-v2 L

In [24]:
imagefolder = "C:/Users/User/images/480_white/"

In [25]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Using device:", device)

model = timm.create_model(
    'tf_efficientnetv2_l.in21k_ft_in1k',
    pretrained=True,
    num_classes=0,       # => no classifier head
    global_pool='avg',   # => global average pooling
    features_only=False  # => return a single (B, 1280) output
)
model = model.to(device)
model.eval()

data_config = timm.data.resolve_data_config({}, model=model)
transform = timm.data.create_transform(**data_config, is_training=False)

def extract_features(images):
    """Run images through the model and return a (batch_size, 1280) tensor."""
    with torch.no_grad():
        # For features_only=False, the model returns a single tensor of shape (B, 1280)
        outputs = model(images)  # shape: (B, 1280)
        # No need to index with [-1] or do any flattening, it is already (B, 1280).
        return outputs

dataset = ImageFolder(imagefolder, transform=transform)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

# Check feature dimension
sample_images, _ = next(iter(dataloader))
sample_images = sample_images.to(device)
sample_features = extract_features(sample_images)
feature_size = sample_features.size(1)
print(f"Sample feature size: {sample_features.shape}")  # Should be (B, 1280)

# Prepare storage for all features/labels
num_samples = len(dataset)
X = torch.zeros(num_samples, feature_size)
y = torch.zeros(num_samples, dtype=torch.long)

start_idx = 0
for images, labels in tqdm(dataloader):
    images = images.to(device)
    labels = labels.to(device)

    features = extract_features(images)  # shape: (batch_size, 1280)
    batch_size = features.size(0)

    # Move features and labels to CPU and store them
    X[start_idx:start_idx+batch_size] = features.cpu()
    y[start_idx:start_idx+batch_size] = labels.cpu()

    start_idx += batch_size

train_test_ratio = 0.8
train_size = round(train_test_ratio * len(X))
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]
print(f"Train size is {train_size}")
print(X_train.shape, X_test.shape)
print(y_train.shape, y_test.shape)

Using device: cuda
Sample feature size: torch.Size([32, 1280])


100%|██████████████████████████████████████████████████████████████████████████████| 1039/1039 [08:34<00:00,  2.02it/s]

Train size is 26586
torch.Size([26586, 1280]) torch.Size([6646, 1280])
torch.Size([26586]) torch.Size([6646])


In [26]:
# Экспорт фичей
X_train_numpy, X_test_numpy = X_train.numpy(), X_test.numpy()
y_train_numpy, y_test_numpy = y_train.numpy(), y_test.numpy()

np.savetxt("X_train_V2_L_480.csv", X_train_numpy, delimiter=",")
np.savetxt("X_test_V2_L_480.csv", X_test_numpy, delimiter=",")
np.savetxt("y_train_V2_L_480.csv", y_train_numpy, delimiter=",")
np.savetxt("y_test_V2_L_480.csv", y_test_numpy, delimiter=",")

In [11]:
# Импорт фичей
X_train_numpy = np.genfromtxt("X_train_V2_L_480.csv", delimiter=",")
X_test_numpy = np.genfromtxt("X_test_V2_L_480.csv", delimiter=",")
y_train_numpy = np.genfromtxt("y_train_V2_L_480.csv", delimiter=",")
y_test_numpy = np.genfromtxt("y_test_V2_L_480.csv", delimiter=",")

In [12]:
print(X_train_numpy.shape)
print(X_test_numpy.shape)
print(y_train_numpy.shape)
print(y_test_numpy.shape)

(26586, 1280)
(6646, 1280)
(26586,)
(6646,)


In [13]:
# Здесь эскпериментируем с модельками
model = CatBoostClassifier(
    iterations=100, 
    loss_function='MultiClass', 
    verbose=False,
    task_type='GPU',
    devices='0'
    )

grid = {'learning_rate': [0.01, 0.05, 0.1],
        'depth': [4, 6, 8, 10],
        'l2_leaf_reg': [1, 3, 5, 7]
       }

grid_search_result = model.grid_search(grid,
                                       X=X_train_numpy,
                                       y=y_train_numpy,
                                       refit=False
                                      )

model = CatBoostClassifier(
    iterations=5000, 
    loss_function='MultiClass', 
    verbose=True,
    task_type='GPU',
    devices='0',
    **grid_search_result['params']
    )

model.fit(X_train_numpy, y_train_numpy)
y_pred = model.predict(X_test_numpy).reshape(-1)
accuracy = np.mean([y_pred == y_test_numpy])
print(f'Best model accuracy: {accuracy}')
print(f'Best model params: {grid_search_result["params"]}')

bestTest = 1.323229609
bestIteration = 99
0:	loss: 1.3232296	best: 1.3232296 (0)	total: 1.57s	remaining: 1m 13s
bestTest = 0.9328818033
bestIteration = 99
1:	loss: 0.9328818	best: 0.9328818 (1)	total: 2.64s	remaining: 1m
bestTest = 0.7936595789
bestIteration = 99
2:	loss: 0.7936596	best: 0.7936596 (2)	total: 3.74s	remaining: 56.1s
bestTest = 1.325086418
bestIteration = 99
3:	loss: 1.3250864	best: 0.7936596 (2)	total: 4.84s	remaining: 53.3s
bestTest = 0.9350932086
bestIteration = 99
4:	loss: 0.9350932	best: 0.7936596 (2)	total: 5.94s	remaining: 51.1s
bestTest = 0.7965791666
bestIteration = 99
5:	loss: 0.7965792	best: 0.7936596 (2)	total: 7.05s	remaining: 49.4s
bestTest = 1.328316346
bestIteration = 99
6:	loss: 1.3283163	best: 0.7936596 (2)	total: 8.21s	remaining: 48.1s
bestTest = 0.9401232033
bestIteration = 99
7:	loss: 0.9401232	best: 0.7936596 (2)	total: 9.31s	remaining: 46.6s
bestTest = 0.8019004038
bestIteration = 99
8:	loss: 0.8019004	best: 0.7936596 (2)	total: 10.4s	remaining: 45.

49:	learn: 0.4423384	total: 13.3s	remaining: 21m 55s
50:	learn: 0.4369870	total: 13.5s	remaining: 21m 53s
51:	learn: 0.4305931	total: 13.8s	remaining: 21m 51s
52:	learn: 0.4230982	total: 14.1s	remaining: 21m 52s
53:	learn: 0.4162621	total: 14.3s	remaining: 21m 52s
54:	learn: 0.4107050	total: 14.6s	remaining: 21m 50s
55:	learn: 0.4048173	total: 14.8s	remaining: 21m 48s
56:	learn: 0.3987827	total: 15.1s	remaining: 21m 48s
57:	learn: 0.3919649	total: 15.4s	remaining: 21m 48s
58:	learn: 0.3866070	total: 15.6s	remaining: 21m 47s
59:	learn: 0.3819373	total: 15.9s	remaining: 21m 45s
60:	learn: 0.3771647	total: 16.1s	remaining: 21m 44s
61:	learn: 0.3724298	total: 16.4s	remaining: 21m 42s
62:	learn: 0.3664158	total: 16.6s	remaining: 21m 43s
63:	learn: 0.3600592	total: 16.9s	remaining: 21m 43s
64:	learn: 0.3541162	total: 17.2s	remaining: 21m 44s
65:	learn: 0.3490116	total: 17.4s	remaining: 21m 44s
66:	learn: 0.3443009	total: 17.7s	remaining: 21m 43s
67:	learn: 0.3395012	total: 18s	remaining: 21m

203:	learn: 0.0888314	total: 51.7s	remaining: 20m 15s
204:	learn: 0.0881573	total: 51.9s	remaining: 20m 14s
205:	learn: 0.0875767	total: 52.2s	remaining: 20m 14s
206:	learn: 0.0869825	total: 52.5s	remaining: 20m 14s
207:	learn: 0.0864085	total: 52.7s	remaining: 20m 14s
208:	learn: 0.0857496	total: 53s	remaining: 20m 14s
209:	learn: 0.0852101	total: 53.2s	remaining: 20m 12s
210:	learn: 0.0845475	total: 53.5s	remaining: 20m 13s
211:	learn: 0.0837558	total: 53.8s	remaining: 20m 14s
212:	learn: 0.0831695	total: 54.1s	remaining: 20m 14s
213:	learn: 0.0826654	total: 54.3s	remaining: 20m 13s
214:	learn: 0.0820343	total: 54.5s	remaining: 20m 13s
215:	learn: 0.0814535	total: 54.8s	remaining: 20m 13s
216:	learn: 0.0807422	total: 55s	remaining: 20m 13s
217:	learn: 0.0800931	total: 55.3s	remaining: 20m 12s
218:	learn: 0.0795420	total: 55.5s	remaining: 20m 11s
219:	learn: 0.0790577	total: 55.7s	remaining: 20m 11s
220:	learn: 0.0787048	total: 55.9s	remaining: 20m 9s
221:	learn: 0.0781084	total: 56.2

355:	learn: 0.0341761	total: 1m 28s	remaining: 19m 17s
356:	learn: 0.0339568	total: 1m 29s	remaining: 19m 17s
357:	learn: 0.0337916	total: 1m 29s	remaining: 19m 17s
358:	learn: 0.0336025	total: 1m 29s	remaining: 19m 16s
359:	learn: 0.0334634	total: 1m 29s	remaining: 19m 16s
360:	learn: 0.0333109	total: 1m 29s	remaining: 19m 15s
361:	learn: 0.0331982	total: 1m 30s	remaining: 19m 14s
362:	learn: 0.0330266	total: 1m 30s	remaining: 19m 14s
363:	learn: 0.0329029	total: 1m 30s	remaining: 19m 13s
364:	learn: 0.0327388	total: 1m 30s	remaining: 19m 13s
365:	learn: 0.0325842	total: 1m 31s	remaining: 19m 12s
366:	learn: 0.0324180	total: 1m 31s	remaining: 19m 12s
367:	learn: 0.0322971	total: 1m 31s	remaining: 19m 11s
368:	learn: 0.0321155	total: 1m 31s	remaining: 19m 11s
369:	learn: 0.0319721	total: 1m 31s	remaining: 19m 10s
370:	learn: 0.0318104	total: 1m 32s	remaining: 19m 10s
371:	learn: 0.0316396	total: 1m 32s	remaining: 19m 10s
372:	learn: 0.0314657	total: 1m 32s	remaining: 19m 10s
373:	learn

507:	learn: 0.0174293	total: 2m 6s	remaining: 18m 36s
508:	learn: 0.0173616	total: 2m 6s	remaining: 18m 36s
509:	learn: 0.0172837	total: 2m 6s	remaining: 18m 36s
510:	learn: 0.0172280	total: 2m 7s	remaining: 18m 35s
511:	learn: 0.0171617	total: 2m 7s	remaining: 18m 35s
512:	learn: 0.0170974	total: 2m 7s	remaining: 18m 35s
513:	learn: 0.0170261	total: 2m 7s	remaining: 18m 35s
514:	learn: 0.0169609	total: 2m 8s	remaining: 18m 34s
515:	learn: 0.0168971	total: 2m 8s	remaining: 18m 34s
516:	learn: 0.0168268	total: 2m 8s	remaining: 18m 34s
517:	learn: 0.0167736	total: 2m 8s	remaining: 18m 34s
518:	learn: 0.0167128	total: 2m 9s	remaining: 18m 33s
519:	learn: 0.0166503	total: 2m 9s	remaining: 18m 33s
520:	learn: 0.0165864	total: 2m 9s	remaining: 18m 33s
521:	learn: 0.0165349	total: 2m 9s	remaining: 18m 33s
522:	learn: 0.0164760	total: 2m 10s	remaining: 18m 33s
523:	learn: 0.0164151	total: 2m 10s	remaining: 18m 32s
524:	learn: 0.0163634	total: 2m 10s	remaining: 18m 32s
525:	learn: 0.0162989	tot

...

4368:	learn: 0.0005928	total: 17m 4s	remaining: 2m 27s
4369:	learn: 0.0005927	total: 17m 4s	remaining: 2m 27s
4370:	learn: 0.0005925	total: 17m 4s	remaining: 2m 27s
4371:	learn: 0.0005923	total: 17m 4s	remaining: 2m 27s
4372:	learn: 0.0005921	total: 17m 5s	remaining: 2m 26s
4373:	learn: 0.0005919	total: 17m 5s	remaining: 2m 26s
4374:	learn: 0.0005917	total: 17m 5s	remaining: 2m 26s
4375:	learn: 0.0005916	total: 17m 5s	remaining: 2m 26s
4376:	learn: 0.0005914	total: 17m 6s	remaining: 2m 26s
4377:	learn: 0.0005912	total: 17m 6s	remaining: 2m 25s
4378:	learn: 0.0005910	total: 17m 6s	remaining: 2m 25s
4379:	learn: 0.0005909	total: 17m 6s	remaining: 2m 25s
4380:	learn: 0.0005907	total: 17m 6s	remaining: 2m 25s
4381:	learn: 0.0005905	total: 17m 7s	remaining: 2m 24s
4382:	learn: 0.0005903	total: 17m 7s	remaining: 2m 24s
4383:	learn: 0.0005901	total: 17m 7s	remaining: 2m 24s
4384:	learn: 0.0005900	total: 17m 7s	remaining: 2m 24s
4385:	learn: 0.0005898	total: 17m 8s	remaining: 2m 23s
4386:	lear

4516:	learn: 0.0005679	total: 17m 37s	remaining: 1m 53s
4517:	learn: 0.0005678	total: 17m 37s	remaining: 1m 52s
4518:	learn: 0.0005677	total: 17m 37s	remaining: 1m 52s
4519:	learn: 0.0005675	total: 17m 37s	remaining: 1m 52s
4520:	learn: 0.0005674	total: 17m 38s	remaining: 1m 52s
4521:	learn: 0.0005672	total: 17m 38s	remaining: 1m 51s
4522:	learn: 0.0005670	total: 17m 38s	remaining: 1m 51s
4523:	learn: 0.0005669	total: 17m 38s	remaining: 1m 51s
4524:	learn: 0.0005668	total: 17m 39s	remaining: 1m 51s
4525:	learn: 0.0005666	total: 17m 39s	remaining: 1m 50s
4526:	learn: 0.0005665	total: 17m 39s	remaining: 1m 50s
4527:	learn: 0.0005663	total: 17m 39s	remaining: 1m 50s
4528:	learn: 0.0005662	total: 17m 39s	remaining: 1m 50s
4529:	learn: 0.0005660	total: 17m 40s	remaining: 1m 49s
4530:	learn: 0.0005659	total: 17m 40s	remaining: 1m 49s
4531:	learn: 0.0005657	total: 17m 40s	remaining: 1m 49s
4532:	learn: 0.0005655	total: 17m 40s	remaining: 1m 49s
4533:	learn: 0.0005653	total: 17m 40s	remaining:

4664:	learn: 0.0005452	total: 18m 11s	remaining: 1m 18s
4665:	learn: 0.0005450	total: 18m 11s	remaining: 1m 18s
4666:	learn: 0.0005449	total: 18m 12s	remaining: 1m 17s
4667:	learn: 0.0005447	total: 18m 12s	remaining: 1m 17s
4668:	learn: 0.0005446	total: 18m 12s	remaining: 1m 17s
4669:	learn: 0.0005445	total: 18m 12s	remaining: 1m 17s
4670:	learn: 0.0005443	total: 18m 12s	remaining: 1m 16s
4671:	learn: 0.0005441	total: 18m 13s	remaining: 1m 16s
4672:	learn: 0.0005440	total: 18m 13s	remaining: 1m 16s
4673:	learn: 0.0005439	total: 18m 13s	remaining: 1m 16s
4674:	learn: 0.0005438	total: 18m 13s	remaining: 1m 16s
4675:	learn: 0.0005436	total: 18m 13s	remaining: 1m 15s
4676:	learn: 0.0005435	total: 18m 14s	remaining: 1m 15s
4677:	learn: 0.0005433	total: 18m 14s	remaining: 1m 15s
4678:	learn: 0.0005432	total: 18m 14s	remaining: 1m 15s
4679:	learn: 0.0005431	total: 18m 14s	remaining: 1m 14s
4680:	learn: 0.0005429	total: 18m 15s	remaining: 1m 14s
4681:	learn: 0.0005427	total: 18m 15s	remaining:

4813:	learn: 0.0005241	total: 18m 46s	remaining: 43.5s
4814:	learn: 0.0005239	total: 18m 46s	remaining: 43.3s
4815:	learn: 0.0005238	total: 18m 46s	remaining: 43.1s
4816:	learn: 0.0005237	total: 18m 47s	remaining: 42.8s
4817:	learn: 0.0005235	total: 18m 47s	remaining: 42.6s
4818:	learn: 0.0005234	total: 18m 47s	remaining: 42.4s
4819:	learn: 0.0005233	total: 18m 47s	remaining: 42.1s
4820:	learn: 0.0005232	total: 18m 47s	remaining: 41.9s
4821:	learn: 0.0005230	total: 18m 48s	remaining: 41.6s
4822:	learn: 0.0005229	total: 18m 48s	remaining: 41.4s
4823:	learn: 0.0005228	total: 18m 48s	remaining: 41.2s
4824:	learn: 0.0005226	total: 18m 48s	remaining: 40.9s
4825:	learn: 0.0005225	total: 18m 49s	remaining: 40.7s
4826:	learn: 0.0005223	total: 18m 49s	remaining: 40.5s
4827:	learn: 0.0005222	total: 18m 49s	remaining: 40.2s
4828:	learn: 0.0005221	total: 18m 49s	remaining: 40s
4829:	learn: 0.0005219	total: 18m 49s	remaining: 39.8s
4830:	learn: 0.0005218	total: 18m 50s	remaining: 39.5s
4831:	learn:

4965:	learn: 0.0005041	total: 19m 20s	remaining: 7.94s
4966:	learn: 0.0005040	total: 19m 20s	remaining: 7.71s
4967:	learn: 0.0005039	total: 19m 20s	remaining: 7.47s
4968:	learn: 0.0005037	total: 19m 20s	remaining: 7.24s
4969:	learn: 0.0005036	total: 19m 21s	remaining: 7.01s
4970:	learn: 0.0005035	total: 19m 21s	remaining: 6.77s
4971:	learn: 0.0005034	total: 19m 21s	remaining: 6.54s
4972:	learn: 0.0005032	total: 19m 21s	remaining: 6.31s
4973:	learn: 0.0005031	total: 19m 21s	remaining: 6.07s
4974:	learn: 0.0005030	total: 19m 22s	remaining: 5.84s
4975:	learn: 0.0005028	total: 19m 22s	remaining: 5.61s
4976:	learn: 0.0005027	total: 19m 22s	remaining: 5.37s
4977:	learn: 0.0005025	total: 19m 22s	remaining: 5.14s
4978:	learn: 0.0005024	total: 19m 23s	remaining: 4.91s
4979:	learn: 0.0005023	total: 19m 23s	remaining: 4.67s
4980:	learn: 0.0005021	total: 19m 23s	remaining: 4.44s
4981:	learn: 0.0005020	total: 19m 23s	remaining: 4.2s
4982:	learn: 0.0005019	total: 19m 24s	remaining: 3.97s
4983:	learn

### EfficientNet-V2 S (384x384)

In [37]:
imagefolder = "C:/Users/User/images/384/"

In [38]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Using device:", device)

model = timm.create_model(
    'tf_efficientnetv2_s.in21k_ft_in1k',
    pretrained=True,
    num_classes=0,       # => no classifier head
    global_pool='avg',   # => global average pooling
    features_only=False  # => return a single (B, 1280) output
)
model = model.to(device)
model.eval()

data_config = timm.data.resolve_data_config({}, model=model)
transform = timm.data.create_transform(**data_config, is_training=False)

def extract_features(images):
    """Run images through the model and return a (batch_size, 1280) tensor."""
    with torch.no_grad():
        # For features_only=False, the model returns a single tensor of shape (B, 1280)
        outputs = model(images)  # shape: (B, 1280)
        # No need to index with [-1] or do any flattening, it is already (B, 1280).
        return outputs

# Example dataset
dataset = ImageFolder(imagefolder, transform=transform)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

# Check feature dimension
sample_images, _ = next(iter(dataloader))
sample_images = sample_images.to(device)
sample_features = extract_features(sample_images)
feature_size = sample_features.size(1)
print(f"Sample feature size: {sample_features.shape}")  # Should be (B, 1280)

# Prepare storage for all features/labels
num_samples = len(dataset)
X = torch.zeros(num_samples, feature_size)
y = torch.zeros(num_samples, dtype=torch.long)

start_idx = 0
for images, labels in tqdm(dataloader):
    images = images.to(device)
    labels = labels.to(device)

    features = extract_features(images)  # shape: (batch_size, 1280)
    batch_size = features.size(0)

    # Move features and labels to CPU and store them
    X[start_idx:start_idx+batch_size] = features.cpu()
    y[start_idx:start_idx+batch_size] = labels.cpu()

    start_idx += batch_size

train_test_ratio = 0.8
train_size = round(train_test_ratio * len(X))
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]
print(f"Train size is {train_size}")
print(X_train.shape, X_test.shape)
print(y_train.shape, y_test.shape)

Using device: cuda
Sample feature size: torch.Size([32, 1280])


100%|██████████████████████████████████████████████████████████████████████████████| 1039/1039 [04:26<00:00,  3.90it/s]

Train size is 26586
torch.Size([26586, 1280]) torch.Size([6646, 1280])
torch.Size([26586]) torch.Size([6646])


In [39]:
# Экспорт фичей
X_train_numpy, X_test_numpy = X_train.numpy(), X_test.numpy()
y_train_numpy, y_test_numpy = y_train.numpy(), y_test.numpy()

np.savetxt("X_train_V2_S_384.csv", X_train_numpy, delimiter=",")
np.savetxt("X_test_V2_S_384.csv", X_test_numpy, delimiter=",")
np.savetxt("y_train_V2_S_384.csv", y_train_numpy, delimiter=",")
np.savetxt("y_test_V2_S_384.csv", y_test_numpy, delimiter=",")

In [14]:
# Импорт фичей
X_train_numpy = np.genfromtxt("X_train_V2_S_384.csv", delimiter=",")
X_test_numpy = np.genfromtxt("X_test_V2_S_384.csv", delimiter=",")
y_train_numpy = np.genfromtxt("y_train_V2_S_384.csv", delimiter=",")
y_test_numpy = np.genfromtxt("y_test_V2_S_384.csv", delimiter=",")

In [15]:
print(X_train_numpy.shape)
print(X_test_numpy.shape)
print(y_train_numpy.shape)
print(y_test_numpy.shape)

(26586, 1280)
(6646, 1280)
(26586,)
(6646,)


In [16]:
# Здесь эскпериментируем с модельками
model = CatBoostClassifier(
    iterations=100, 
    loss_function='MultiClass', 
    verbose=False,
    task_type='GPU',
    devices='0'
    )

grid = {'learning_rate': [0.01, 0.05, 0.1],
        'depth': [4, 6, 8, 10],
        'l2_leaf_reg': [1, 3, 5, 7]
       }

grid_search_result = model.grid_search(grid,
                                       X=X_train_numpy,
                                       y=y_train_numpy,
                                       refit=False
                                      )

model = CatBoostClassifier(
    iterations=5000, 
    loss_function='MultiClass', 
    verbose=True,
    task_type='GPU',
    devices='0',
    **grid_search_result['params']
    )

model.fit(X_train_numpy, y_train_numpy)
y_pred = model.predict(X_test_numpy).reshape(-1)
accuracy = np.mean([y_pred == y_test_numpy])
print(f'Best model accuracy: {accuracy}')
print(f'Best model params: {grid_search_result["params"]}')

bestTest = 1.275752107
bestIteration = 99
0:	loss: 1.2757521	best: 1.2757521 (0)	total: 1.55s	remaining: 1m 12s
bestTest = 0.8840209915
bestIteration = 99
1:	loss: 0.8840210	best: 0.8840210 (1)	total: 2.64s	remaining: 1m
bestTest = 0.7506163196
bestIteration = 99
2:	loss: 0.7506163	best: 0.7506163 (2)	total: 3.73s	remaining: 56s
bestTest = 1.277712486
bestIteration = 99
3:	loss: 1.2777125	best: 0.7506163 (2)	total: 4.87s	remaining: 53.5s
bestTest = 0.8878134806
bestIteration = 99
4:	loss: 0.8878135	best: 0.7506163 (2)	total: 5.98s	remaining: 51.4s
bestTest = 0.758203731
bestIteration = 99
5:	loss: 0.7582037	best: 0.7506163 (2)	total: 7.1s	remaining: 49.7s
bestTest = 1.279547994
bestIteration = 99
6:	loss: 1.2795480	best: 0.7506163 (2)	total: 8.23s	remaining: 48.2s
bestTest = 0.8950871083
bestIteration = 99
7:	loss: 0.8950871	best: 0.7506163 (2)	total: 9.33s	remaining: 46.7s
bestTest = 0.7642779571
bestIteration = 99
8:	loss: 0.7642780	best: 0.7506163 (2)	total: 10.5s	remaining: 45.4s
b

49:	learn: 0.4652495	total: 13.4s	remaining: 22m 6s
50:	learn: 0.4576620	total: 13.7s	remaining: 22m 6s
51:	learn: 0.4521122	total: 13.9s	remaining: 22m 4s
52:	learn: 0.4461477	total: 14.2s	remaining: 22m 3s
53:	learn: 0.4400428	total: 14.4s	remaining: 22m 3s
54:	learn: 0.4336769	total: 14.7s	remaining: 22m 2s
55:	learn: 0.4284897	total: 14.9s	remaining: 21m 59s
56:	learn: 0.4220682	total: 15.2s	remaining: 22m
57:	learn: 0.4166626	total: 15.5s	remaining: 21m 58s
58:	learn: 0.4102036	total: 15.7s	remaining: 21m 58s
59:	learn: 0.4053991	total: 16s	remaining: 21m 56s
60:	learn: 0.3994385	total: 16.3s	remaining: 21m 57s
61:	learn: 0.3943266	total: 16.5s	remaining: 21m 56s
62:	learn: 0.3892714	total: 16.8s	remaining: 21m 55s
63:	learn: 0.3838000	total: 17.1s	remaining: 21m 55s
64:	learn: 0.3794046	total: 17.3s	remaining: 21m 54s
65:	learn: 0.3745118	total: 17.6s	remaining: 21m 54s
66:	learn: 0.3698996	total: 17.8s	remaining: 21m 52s
67:	learn: 0.3645496	total: 18.1s	remaining: 21m 52s
68:	l

203:	learn: 0.1055108	total: 57.8s	remaining: 22m 39s
204:	learn: 0.1048828	total: 58.1s	remaining: 22m 39s
205:	learn: 0.1041329	total: 58.4s	remaining: 22m 38s
206:	learn: 0.1036377	total: 58.6s	remaining: 22m 37s
207:	learn: 0.1027635	total: 58.9s	remaining: 22m 37s
208:	learn: 0.1019312	total: 59.2s	remaining: 22m 36s
209:	learn: 0.1012631	total: 59.4s	remaining: 22m 35s
210:	learn: 0.1007109	total: 59.6s	remaining: 22m 32s
211:	learn: 0.0997398	total: 59.9s	remaining: 22m 32s
212:	learn: 0.0992139	total: 1m	remaining: 22m 30s
213:	learn: 0.0984847	total: 1m	remaining: 22m 29s
214:	learn: 0.0977890	total: 1m	remaining: 22m 28s
215:	learn: 0.0972162	total: 1m	remaining: 22m 28s
216:	learn: 0.0965167	total: 1m 1s	remaining: 22m 27s
217:	learn: 0.0958542	total: 1m 1s	remaining: 22m 27s
218:	learn: 0.0955044	total: 1m 1s	remaining: 22m 25s
219:	learn: 0.0947614	total: 1m 1s	remaining: 22m 25s
220:	learn: 0.0940679	total: 1m 2s	remaining: 22m 25s
221:	learn: 0.0935582	total: 1m 2s	remai

354:	learn: 0.0451539	total: 1m 34s	remaining: 20m 35s
355:	learn: 0.0448684	total: 1m 34s	remaining: 20m 34s
356:	learn: 0.0447207	total: 1m 34s	remaining: 20m 33s
357:	learn: 0.0444481	total: 1m 35s	remaining: 20m 32s
358:	learn: 0.0442027	total: 1m 35s	remaining: 20m 32s
359:	learn: 0.0440100	total: 1m 35s	remaining: 20m 31s
360:	learn: 0.0436924	total: 1m 35s	remaining: 20m 31s
361:	learn: 0.0434485	total: 1m 36s	remaining: 20m 31s
362:	learn: 0.0432672	total: 1m 36s	remaining: 20m 30s
363:	learn: 0.0430760	total: 1m 36s	remaining: 20m 29s
364:	learn: 0.0428585	total: 1m 36s	remaining: 20m 28s
365:	learn: 0.0426678	total: 1m 36s	remaining: 20m 28s
366:	learn: 0.0424157	total: 1m 37s	remaining: 20m 27s
367:	learn: 0.0422374	total: 1m 37s	remaining: 20m 26s
368:	learn: 0.0420020	total: 1m 37s	remaining: 20m 26s
369:	learn: 0.0417660	total: 1m 37s	remaining: 20m 25s
370:	learn: 0.0416251	total: 1m 38s	remaining: 20m 24s
371:	learn: 0.0414349	total: 1m 38s	remaining: 20m 23s
372:	learn

505:	learn: 0.0232047	total: 2m 13s	remaining: 19m 49s
506:	learn: 0.0231169	total: 2m 14s	remaining: 19m 49s
507:	learn: 0.0229937	total: 2m 14s	remaining: 19m 50s
508:	learn: 0.0228834	total: 2m 14s	remaining: 19m 50s
509:	learn: 0.0227942	total: 2m 15s	remaining: 19m 49s
510:	learn: 0.0227094	total: 2m 15s	remaining: 19m 49s
511:	learn: 0.0226623	total: 2m 15s	remaining: 19m 49s
512:	learn: 0.0225747	total: 2m 15s	remaining: 19m 49s
513:	learn: 0.0224912	total: 2m 16s	remaining: 19m 48s
514:	learn: 0.0224069	total: 2m 16s	remaining: 19m 48s
515:	learn: 0.0223409	total: 2m 16s	remaining: 19m 48s
516:	learn: 0.0222719	total: 2m 17s	remaining: 19m 48s
517:	learn: 0.0222058	total: 2m 17s	remaining: 19m 47s
518:	learn: 0.0221130	total: 2m 17s	remaining: 19m 47s
519:	learn: 0.0220350	total: 2m 17s	remaining: 19m 47s
520:	learn: 0.0219473	total: 2m 18s	remaining: 19m 47s
521:	learn: 0.0218792	total: 2m 18s	remaining: 19m 47s
522:	learn: 0.0217857	total: 2m 18s	remaining: 19m 46s
523:	learn

...

4360:	learn: 0.0006753	total: 17m 19s	remaining: 2m 32s
4361:	learn: 0.0006751	total: 17m 19s	remaining: 2m 32s
4362:	learn: 0.0006749	total: 17m 19s	remaining: 2m 31s
4363:	learn: 0.0006747	total: 17m 19s	remaining: 2m 31s
4364:	learn: 0.0006745	total: 17m 19s	remaining: 2m 31s
4365:	learn: 0.0006743	total: 17m 20s	remaining: 2m 31s
4366:	learn: 0.0006741	total: 17m 20s	remaining: 2m 30s
4367:	learn: 0.0006739	total: 17m 20s	remaining: 2m 30s
4368:	learn: 0.0006737	total: 17m 20s	remaining: 2m 30s
4369:	learn: 0.0006735	total: 17m 20s	remaining: 2m 30s
4370:	learn: 0.0006732	total: 17m 21s	remaining: 2m 29s
4371:	learn: 0.0006731	total: 17m 21s	remaining: 2m 29s
4372:	learn: 0.0006729	total: 17m 21s	remaining: 2m 29s
4373:	learn: 0.0006727	total: 17m 21s	remaining: 2m 29s
4374:	learn: 0.0006725	total: 17m 22s	remaining: 2m 28s
4375:	learn: 0.0006723	total: 17m 22s	remaining: 2m 28s
4376:	learn: 0.0006721	total: 17m 22s	remaining: 2m 28s
4377:	learn: 0.0006720	total: 17m 22s	remaining:

4509:	learn: 0.0006463	total: 17m 53s	remaining: 1m 56s
4510:	learn: 0.0006461	total: 17m 53s	remaining: 1m 56s
4511:	learn: 0.0006460	total: 17m 53s	remaining: 1m 56s
4512:	learn: 0.0006458	total: 17m 54s	remaining: 1m 55s
4513:	learn: 0.0006456	total: 17m 54s	remaining: 1m 55s
4514:	learn: 0.0006455	total: 17m 54s	remaining: 1m 55s
4515:	learn: 0.0006453	total: 17m 54s	remaining: 1m 55s
4516:	learn: 0.0006451	total: 17m 55s	remaining: 1m 54s
4517:	learn: 0.0006448	total: 17m 55s	remaining: 1m 54s
4518:	learn: 0.0006447	total: 17m 55s	remaining: 1m 54s
4519:	learn: 0.0006445	total: 17m 55s	remaining: 1m 54s
4520:	learn: 0.0006443	total: 17m 55s	remaining: 1m 53s
4521:	learn: 0.0006441	total: 17m 56s	remaining: 1m 53s
4522:	learn: 0.0006440	total: 17m 56s	remaining: 1m 53s
4523:	learn: 0.0006438	total: 17m 56s	remaining: 1m 53s
4524:	learn: 0.0006436	total: 17m 56s	remaining: 1m 53s
4525:	learn: 0.0006434	total: 17m 57s	remaining: 1m 52s
4526:	learn: 0.0006432	total: 17m 57s	remaining:

4657:	learn: 0.0006195	total: 18m 26s	remaining: 1m 21s
4658:	learn: 0.0006193	total: 18m 26s	remaining: 1m 20s
4659:	learn: 0.0006191	total: 18m 26s	remaining: 1m 20s
4660:	learn: 0.0006189	total: 18m 26s	remaining: 1m 20s
4661:	learn: 0.0006187	total: 18m 27s	remaining: 1m 20s
4662:	learn: 0.0006186	total: 18m 27s	remaining: 1m 20s
4663:	learn: 0.0006184	total: 18m 27s	remaining: 1m 19s
4664:	learn: 0.0006182	total: 18m 27s	remaining: 1m 19s
4665:	learn: 0.0006181	total: 18m 28s	remaining: 1m 19s
4666:	learn: 0.0006179	total: 18m 28s	remaining: 1m 19s
4667:	learn: 0.0006177	total: 18m 28s	remaining: 1m 18s
4668:	learn: 0.0006175	total: 18m 28s	remaining: 1m 18s
4669:	learn: 0.0006174	total: 18m 28s	remaining: 1m 18s
4670:	learn: 0.0006172	total: 18m 29s	remaining: 1m 18s
4671:	learn: 0.0006170	total: 18m 29s	remaining: 1m 17s
4672:	learn: 0.0006168	total: 18m 29s	remaining: 1m 17s
4673:	learn: 0.0006167	total: 18m 29s	remaining: 1m 17s
4674:	learn: 0.0006165	total: 18m 30s	remaining:

4807:	learn: 0.0005942	total: 18m 59s	remaining: 45.5s
4808:	learn: 0.0005940	total: 19m	remaining: 45.3s
4809:	learn: 0.0005939	total: 19m	remaining: 45s
4810:	learn: 0.0005937	total: 19m	remaining: 44.8s
4811:	learn: 0.0005936	total: 19m	remaining: 44.6s
4812:	learn: 0.0005934	total: 19m	remaining: 44.3s
4813:	learn: 0.0005932	total: 19m 1s	remaining: 44.1s
4814:	learn: 0.0005931	total: 19m 1s	remaining: 43.9s
4815:	learn: 0.0005930	total: 19m 1s	remaining: 43.6s
4816:	learn: 0.0005928	total: 19m 1s	remaining: 43.4s
4817:	learn: 0.0005927	total: 19m 2s	remaining: 43.1s
4818:	learn: 0.0005925	total: 19m 2s	remaining: 42.9s
4819:	learn: 0.0005924	total: 19m 2s	remaining: 42.7s
4820:	learn: 0.0005922	total: 19m 2s	remaining: 42.4s
4821:	learn: 0.0005920	total: 19m 2s	remaining: 42.2s
4822:	learn: 0.0005919	total: 19m 3s	remaining: 42s
4823:	learn: 0.0005917	total: 19m 3s	remaining: 41.7s
4824:	learn: 0.0005915	total: 19m 3s	remaining: 41.5s
4825:	learn: 0.0005913	total: 19m 3s	remaining

4958:	learn: 0.0005709	total: 19m 34s	remaining: 9.71s
4959:	learn: 0.0005707	total: 19m 34s	remaining: 9.47s
4960:	learn: 0.0005705	total: 19m 34s	remaining: 9.23s
4961:	learn: 0.0005704	total: 19m 34s	remaining: 9s
4962:	learn: 0.0005703	total: 19m 35s	remaining: 8.76s
4963:	learn: 0.0005701	total: 19m 35s	remaining: 8.52s
4964:	learn: 0.0005700	total: 19m 35s	remaining: 8.29s
4965:	learn: 0.0005699	total: 19m 35s	remaining: 8.05s
4966:	learn: 0.0005697	total: 19m 36s	remaining: 7.82s
4967:	learn: 0.0005696	total: 19m 36s	remaining: 7.58s
4968:	learn: 0.0005694	total: 19m 36s	remaining: 7.34s
4969:	learn: 0.0005692	total: 19m 37s	remaining: 7.11s
4970:	learn: 0.0005691	total: 19m 37s	remaining: 6.87s
4971:	learn: 0.0005689	total: 19m 37s	remaining: 6.63s
4972:	learn: 0.0005687	total: 19m 37s	remaining: 6.39s
4973:	learn: 0.0005686	total: 19m 38s	remaining: 6.16s
4974:	learn: 0.0005685	total: 19m 38s	remaining: 5.92s
4975:	learn: 0.0005683	total: 19m 38s	remaining: 5.68s
4976:	learn: 